# 1. What is PyTorch?

PyTorch is an open-source machine learning library developed by Facebook's AI Research lab.

It provides a flexible and dynamic computational graph, which makes it particularly suited for research and prototyping.

PyTorch is known for its simplicity and ease of use, making it a popular choice for both beginners and experienced researchers in the field of deep learning.

**Key Features**:

  * **Dynamic Computational Graph**: A **dynamic computation graph is constructed in real-time during runtime**, while a static graph is pre-defined before execution. The adaptability of dynamic graphs facilitates complex architectures and simplifies debugging, whereas static graphs can optimize performance but may lack flexibility for certain model structures.
  * **Tensors**: PyTorch operates on multi-dimensional arrays called tensors, which are similar to NumPy's ndarrays but **with the ability to run on GPUs**.
  * **Neural Network Module**: PyTorch **provides torch.nn module** to define and train neural networks.
  * **GPU Acceleration**: PyTorch **supports GPU acceleration**, allowing computations to be performed much faster than on a CPU.

# 2. Why use PyTorch?

While NumPy is an essential tool for numerical operations in Python, PyTorch offers additional features tailored for deep learning and neural network research:

**Autograd: Automatic Differentiation**

* The most significant advantage of PyTorch over NumPy is its autograd system. **Autograd automatically computes the gradients or derivatives of operations**, which is essential for training neural networks using gradient-based optimization algorithms.

* In deep learning, we often need to calculate the gradient of a loss function with respect to model parameters. **Manual computation can be error-prone and tedious**. Autograd simplifies this by automatically calculating gradients.

##  Example:

Let's say we have a simple operation like $y = x^2$ as an example.

NumPy (without Autograd)

In [1]:
import numpy as np

# Define a value
#임의의 배열(array)를 설정
x_np = np.array([2.0])

# Manually compute the gradient
dy_dx = 2 * x_np

print(dy_dx)

[4.]


PyTorch (with Autograd)

In [2]:
## y.backward()의 의미는 다음으로 생각할 수 있다.
## Forward pass(정방향)의 계산을 진행할 때, x_pt -> y 순으로 계산이 이루어 짐을 알 수 있다.
## Backward pass(역방향)의 계산, 즉 gradient 계산을 진행한다면, 계산의 순서는 y -> x_pt가 되어 y.backward()의 계산이 x_pt.grad가 되는 것이다.

## 쉽게 말해 backward()는 역방향 계산(gradient)을 의미하고
## .grad는 이로 계산된 최초 tensor의 gradient를 계산, 축적하여 반환한다.

## 여기서 requires_grad=True는 grad가 축적되게 하라는 의미인데, 다른 말로 backward가 계속 주시하게 하라는 말이며, 지속해서 업데이트 된다는 말이다.
## 반면 이 옵션이 False라면 grad가 축적, 즉 업데이트 되지 않는다는 의미이고, 그 말은 다시 변하지 않는 상수라고 할 수 있겠다.

## 정방향의 전파만을 수행하거나 텐서가 상수라면 requires_grad를 해제하거나 False로 설정하는 것이 바람직하다.

In [3]:
import torch

# Define a tensor and set requires_grad=True to track computation with it
x_pt = torch.tensor([2.0], requires_grad=True)

# Define an operation
y = x_pt ** 2

# Compute gradients
y.backward()

# Display gradient
print(x_pt.grad)

tensor([4.])


While this is a simple example, imagine having to compute gradients for complex operations and architectures manually.

**Disabling Gradient Tracking**

By default, all tensors with **requires_grad=True** are tracking their computational history and support gradient computation. However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e. we only want to do forward computations through the network. We can stop tracking computations by surrounding our computation code with **torch.no_grad()** block:

In [4]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)  # weight
b = torch.randn(3, requires_grad=True)  # bias

In [5]:
z = torch.matmul(x, w)+b    # 다차원 텐서 곱, 텐서의 행렬 계산
print(z.requires_grad)  # requires_grad 여부 확인

with torch.no_grad():   # default인 requires_grad를 해제. with는 -일 때 정도로 해석
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the **detach()** method on the tensor:

In [6]:
z = torch.matmul(x, w)+b
z_det = z.detach()  # no_grad()와 동일
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:
- To mark some parameters in your neural network as frozen parameters.
- To speed up computations when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.

# 3. Handling **torch.Tensor**

In PyTorch, the fundamental object used for almost all computational tasks is the Tensor. A **Tensor** is similar to NumPy's **ndarray** but with the **added capability to be used on a GPU for faster computations**. Tensors are multi-dimensional arrays and are at the core of PyTorch's design.

## a. Creating Tensors

From Lists

In [7]:
data = [[1, 2], [3, 4]]
tensor_from_data = torch.tensor(data)

print(tensor_from_data)

tensor([[1, 2],
        [3, 4]])


From NumPy Arrays

In [37]:
np_array = np.array(data)
print(np_array)

tensor_from_numpy = torch.from_numpy(np_array)
print(tensor_from_numpy)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


Using Built-in Functions

In [9]:
tensor_zeros = torch.zeros(3, 3)
tensor_ones = torch.ones(3, 3)
tensor_eye = torch.eye(3)  # Identity matrix
tensor_rand = torch.rand(3, 3)  # Uniform random numbers between 0 and 1

print(tensor_zeros)
print(tensor_ones)
print(tensor_eye)
print(tensor_rand)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[0.0611, 0.9114, 0.0145],
        [0.9530, 0.2804, 0.8591],
        [0.0473, 0.6393, 0.8411]])


## b. Tensor Operations

**Arithmetic Operations**

Arithmetic operations in PyTorch are element-wise operations similar to those in NumPy.

In [10]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])

# Element-wise addition
z1 = x + y  # or torch.add(x, y)
print(z1)
# Output
# tensor([5, 7, 9])

# Element-wise subtraction
z2 = x - y  # or torch.sub(x, y)
print(z2)
# Output
# tensor([-3, -3, -3])

# Element-wise multiplication
z3 = x * y  # or torch.mul(x, y)
print(z3)
# Output
# tensor([4, 10, 18])

# Element-wise division
z4 = x / y  # or torch.div(x, y)
print(z4)
# Output
# tensor([0.2500, 0.4000, 0.5000])

tensor([5, 7, 9])
tensor([-3, -3, -3])
tensor([ 4, 10, 18])
tensor([0.2500, 0.4000, 0.5000])


**Reduction Operations**

Reduction operations reduce the number of elements in a tensor.

In [11]:
x = torch.tensor([1, 2, 3, 4])

# Sum of all elements
sum_all = torch.sum(x)
print(sum_all)
# Output
# tensor(10)

# Mean of all elements (for float tensor)
mean_all = torch.mean(x.float())
print(mean_all)
# Output
# tensor(2.5000)

# Max and Min values
max_val = torch.max(x).item() # .item = tensor에서 값만 가져오기
min_val = torch.min(x).item()
print(max_val, min_val)
# Output
# 4 1

tensor(10)
tensor(2.5000)
4 1


**Matrix Operations**

Matrix operations are fundamental in deep learning, especially in neural network layers.



In [12]:
mat1 = torch.tensor([[1, 2], [3, 4]])
mat2 = torch.tensor([[2, 1], [1, 2]])

# Matrix multiplication
matmul_result = torch.mm(mat1, mat2)    # .matmul()과의 차이점: broadcast가 안됨(matrix의 사이즈가 같아야 함)
print(matmul_result)

# Element-wise matrix multiplication
elementwise_mul = mat1 * mat2
print(elementwise_mul)

# Matrix transpose
transpose_mat = torch.t(mat1)
print(transpose_mat)

tensor([[ 4,  5],
        [10, 11]])
tensor([[2, 2],
        [3, 8]])
tensor([[1, 3],
        [2, 4]])


**Reshaping**

Reshaping allows you to change the shape (number of dimensions and size along each dimension) of a tensor.



In [13]:
x = torch.tensor([[1, 2], [3, 4]])

# Reshape to 4x1 tensor
# This stacks the rows of 'x' into a single column.
reshaped = x.view(4, 1)     # contiguous(axis 방향으로 자료가 저장된 상태) 성질을 만족해야 사용 가능
print(reshaped)
print(reshaped.shape)

# Flatten the tensor
# This transforms 'x' into a 1D tensor by unrolling its values.
# The '-1' in view indicates to infer the size for that dimension based on the original tensor.
flattened = x.view(-1)
print(flattened)
print(flattened.shape)

tensor([[1],
        [2],
        [3],
        [4]])
torch.Size([4, 1])
tensor([1, 2, 3, 4])
torch.Size([4])


The reshape function is similar to **view** but provides more flexibility as it returns a new tensor with the desired shape. If the requested shape is compatible with the original tensor and no memory copy is needed, it will share the same data; otherwise, a copy will be made.

**Squeeze** and **Unsqueeze**

**squeeze** function removes dimensions of size 1 from a tensor's shape. It's useful to reduce unnecessary dimensions.



In [14]:
# .squeeze() = 차원이 1인 경우 해당 차원을 제거

x = torch.tensor([[1], [2], [3]])
print(x.shape)

squeezed_tensor = x.squeeze()
print(squeezed_tensor.shape)

print(squeezed_tensor)

# 2차원 벡터에서 1차원 벡터로 변경됨을 알 수 있다.

torch.Size([3, 1])
torch.Size([3])
tensor([1, 2, 3])



 As the opposite of squeeze, **unsqueeze** function adds a dimension of size 1 to a tensor's shape at a specified position.

In [15]:
# 하지만 필요시에 나타나게 만들 수 있다.

x = torch.tensor([1, 2, 3])
print(x.shape)

print()

unsqueezed_tensor1 = x.unsqueeze(dim=0)     # 괄호 안에는 생성할 차원(Dim)을 정해주어야 한다.
print(unsqueezed_tensor1.shape)

print(unsqueezed_tensor1)

print()

unsqueezed_tensor2 = x.unsqueeze(dim=1)     # Dim=0: 행, Dim=1: 열
print(unsqueezed_tensor2.shape)

print(unsqueezed_tensor2)

torch.Size([3])

torch.Size([1, 3])
tensor([[1, 2, 3]])

torch.Size([3, 1])
tensor([[1],
        [2],
        [3]])


In [16]:
x_reshape = torch.tensor([1, 2, 3, 4, 5, 6])
reshaped_tensor = x_reshape.reshape(2, 3)
print(reshaped_tensor)
print(reshaped_tensor.shape)

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])


**Concatenation**

You can concatenate multiple tensors along a specific dimension.

In [17]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])

# Concatenate along dimension 0
concatenated_dim0 = torch.cat([x, y], dim=0)    # 0차원(행)에 대해서 결합
print(concatenated_dim0)
# Output
# tensor([1, 2, 3, 4, 5, 6])
print(concatenated_dim0.shape)

# If you have 2D tensors, you can concatenate along dimension 1 as well
x2 = x.unsqueeze(1)
y2 = y.unsqueeze(1)
print(x2, y2, sep='\n')
print(x2.shape, y2.shape, sep='\n')

concatenated_dim1 = torch.cat([x2, y2], dim=1)
print(concatenated_dim1)
print(concatenated_dim1.shape)

tensor([1, 2, 3, 4, 5, 6])
torch.Size([6])
tensor([[1],
        [2],
        [3]])
tensor([[4],
        [5],
        [6]])
torch.Size([3, 1])
torch.Size([3, 1])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
torch.Size([3, 2])


**Indexing, Slicing, Joining, and Mutating**

These operations allow you to access and modify specific parts of a tensor.



In [18]:
x = torch.tensor([10, 20, 30, 40, 50])

# Indexing: Get the second element
second_element = x[1]
print(second_element)
# Output
# tensor(20)

# Slicing: Get the second and third elements
slice_2_3 = x[1:3]
print(slice_2_3)
# Output
# tensor([20, 30])

# Joining: Stack tensors together
stacked = torch.stack([x, x])
print(stacked)
# Output
# tensor([[10, 20, 30, 40, 50],
#         [10, 20, 30, 40, 50]])

# Mutating: Change the first element of x to 100
x[0] = 100
print(x)
# Output
# tensor([100,  20,  30,  40,  50])

tensor(20)
tensor([20, 30])
tensor([[10, 20, 30, 40, 50],
        [10, 20, 30, 40, 50]])
tensor([100,  20,  30,  40,  50])


## c. Broadcasting

PyTorch supports broadcasting, a feature borrowed from NumPy. It allows PyTorch to work with arrays of different shapes when performing arithmetic operations.

### Examples

**Scalar and Tensor**

Adding a scalar to a tensor broadcasts the scalar across all elements of the tensor.

In [19]:
tensor = torch.tensor([1, 2, 3])
result = tensor + 2
print(result)

tensor([3, 4, 5])


**Tensors with Different Dimensions**

Adding a tensor of shape [3, 1] to a tensor of shape [3] will broadcast the second tensor across the rows of the first one.

In [20]:
tensor_a = torch.tensor([[1], [2], [3]])
tensor_b = torch.tensor([4, 5, 6])
print(tensor_a.shape, tensor_b.shape)

result = tensor_a + tensor_b
print(result)
print(result.shape)


torch.Size([3, 1]) torch.Size([3])
tensor([[5, 6, 7],
        [6, 7, 8],
        [7, 8, 9]])
torch.Size([3, 3])


**Tensors with Mismatched Dimensions**

Operations between tensors with mismatched dimensions where neither dimension is 1 will raise an error.

In [21]:
tensor_c = torch.tensor([[1, 2], [3, 4]])
tensor_d = torch.tensor([1, 2, 3])
print(tensor_c.shape, tensor_d.shape)

# This will raise an error: result = tensor_c + tensor_d
try:
    result = tensor_c + tensor_d
except RuntimeError as e:
    error_message = str(e)
    print(error_message)

torch.Size([2, 2]) torch.Size([3])
The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1


## d. Moving Tensors between CPU and GPU

In [22]:
import torch
print(torch.cuda.is_available())

True


**Moving a Tensor to GPU**

PyTorch uses CUDA (a parallel computing platform and application programming interface model created by NVIDIA) to enable GPU computations. To move a tensor to the GPU, the cuda() method is used.

In [23]:
# Create a tensor on CPU
tensor_cpu = torch.tensor([1, 2, 3])
print(tensor_cpu.device)

# Move the tensor to GPU (if CUDA is available)
if torch.cuda.is_available():
    tensor_gpu = tensor_cpu.cuda()
    print(tensor_gpu.device)

cpu
cuda:0


**Moving a Tensor to a Specific GPU**

In systems with multiple GPUs, they are numbered as 0, 1, 2, etc. You can select a specific GPU by setting its ID. Once the device is set to a specific GPU ID, you can move the tensor to that GPU using the to() method.

In [24]:
# Set the GPU ID (for this example, we'll set it to 0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Move the tensor to the specified GPU
tensor_gpu_specific = tensor_cpu.to(device)
print(tensor_gpu_specific.device)

cuda:0
cuda:0


**Moving a Tensor back to CPU**

Once the computations on the GPU are completed, you might want to move the results back to the CPU. This can be achieved using the cpu() method.

In [25]:
# Move the tensor back to CPU from GPU
tensor_cpu_again = tensor_gpu.cpu()
print(tensor_cpu_again.device)

cpu


If the tensors are not on the same device, you cannot perform operations on them.

In [26]:
# Assuming you have a CUDA-enabled GPU
if torch.cuda.is_available():
    # Create a tensor on the CPU
    tensor_cpu = torch.tensor([1, 2, 3])

    # Move the tensor to GPU
    tensor_gpu = tensor_cpu.cuda()

    # Attempt to add the CPU tensor and GPU tensor
    # This will raise an error because the tensors are on different devices
    try:
        result = tensor_cpu + tensor_gpu
    except RuntimeError as e:
        error_message = str(e)
        print(error_message)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


# 4. Exercise

1. Create two tensors of size (3, 3) with random values. Multiply them element-wise.

In [27]:
# Write your code here


2. Create a tensor **A** of size (5, 3) and another tensor **B** of size (3,). Add them.

In [28]:
# Write your code here


3. Create two tensors of size (4, 3) and (3, 2). Perform matrix multiplication.

In [29]:
# Write your code here


4. Create a tensor of size (2, 6). Reshape it to (3, 4).

In [30]:
# Write your code here


5. Create a tensor **A** of size (5, 1) and another tensor **B** of size (5,). Perform element-wise multiplication.

In [31]:
# Write your code here


6. Create two tensors of size (3, 4). Transpose the second tensor and perform matrix multiplication.

In [32]:
# Write your code here


7. Create a tensor of size (3, 1) and expand its size to (3, 4). Sum the elements along the second dimension.

In [33]:
# Write your code here


8. Create a tensor of size (6, 5). Calculate the mean along the second dimension.

In [34]:
# Write your code here


9. Create a tensor of size (4, 4). Extract the second and fourth row.

In [35]:
# Write your code here


10. Calculate the outer product of two tensors of size (5,) and (3,).

- *Search for the outer product function on Google!*

In [36]:
# Write your code here
